In [1]:
import logging
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

In [2]:
kite = KiteConnect(api_key="37wk1ehie3noonc1")

In [3]:
print(kite.login_url())

https://kite.trade/connect/login?api_key=37wk1ehie3noonc1&v=3


In [4]:
data = kite.generate_session("UK2Uozain7Bn0aRDwuvj71uSKVpA1Ksz", api_secret="oqzjzmrz6674x4xl7mbfq0qltd1oekts")

In [5]:
kite.set_access_token(data["access_token"])


In [6]:
instruments = kite.instruments(exchange='NSE')

In [7]:
instruments[0]

OrderedDict([('instrument_token', 259081),
             ('exchange_token', '1012'),
             ('tradingsymbol', 'NIFTY50 PR 1X INV'),
             ('name', 'NIFTY50 PR 1X INV'),
             ('last_price', 0.0),
             ('expiry', ''),
             ('strike', 0.0),
             ('tick_size', 0.0),
             ('lot_size', 0),
             ('instrument_type', 'EQ'),
             ('segment', 'INDICES'),
             ('exchange', 'NSE')])

In [8]:
for instrument in instruments:
    if instrument['tradingsymbol'] == 'BPCL':
        print(instrument['instrument_token'])

134657


In [9]:
for instrument in instruments:
    if instrument['tradingsymbol'] == 'INFY':
        print(instrument['instrument_token'])

408065


In [39]:
# logging.basicConfig(level=logging.DEBUG)

In [12]:
# Initialise
kws = KiteTicker("37wk1ehie3noonc1", data['access_token'])
tick_count = 0

In [13]:
def on_ticks(ws, ticks):
    # Callback to receive ticks.
    global tick_count
    print("Ticks: {}".format(ticks))
    tick_count = tick_count + 1
    print("Length: {}".format(tick_count))
    if tick_count > 5:
        ws.close()

def on_connect(ws, response):
    pass
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([134657, 408065])
#     ws.subscribe([134657])
    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_FULL, [408065])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

In [14]:
# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

In [15]:
# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()

Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 408065, 'last_price': 763.65, 'last_quantity': 32, 'average_price': 759.03, 'volume': 5773332, 'buy_quantity': 1191083, 'sell_quantity': 1103674, 'ohlc': {'open': 760.0, 'high': 764.0, 'low': 755.0, 'close': 738.15}, 'change': 3.454582401950823}, {'tradable': True, 'mode': 'quote', 'instrument_token': 134657, 'last_price': 472.1, 'last_quantity': 5, 'average_price': 472.01, 'volume': 198183, 'buy_quantity': 253002, 'sell_quantity': 285802, 'ohlc': {'open': 471.0, 'high': 474.0, 'low': 470.9, 'close': 470.25}, 'change': 0.3934077618288193}]
Length: 1
Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 134657, 'last_price': 472.1, 'last_quantity': 13, 'average_price': 472.01, 'volume': 198434, 'buy_quantity': 253323, 'sell_quantity': 285435, 'ohlc': {'open': 471.0, 'high': 474.0, 'low': 470.9, 'close': 470.25}, 'change': 0.3934077618288193}, {'tradable': True, 'mode': 'quote', 'instrument_token': 408065, 'last_pr

Connection closed: None - None


Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 134657, 'last_price': 472.1, 'last_quantity': 5, 'average_price': 472.01, 'volume': 198439, 'buy_quantity': 253323, 'sell_quantity': 285645, 'ohlc': {'open': 471.0, 'high': 474.0, 'low': 470.9, 'close': 470.25}, 'change': 0.3934077618288193}]
Length: 6
